# **LTI&trade; Advantage** bootcamp notebook for Tool

## Introduction

The notebook shows how to interact with the LTI Advantage ecosystem from a tool implementer viewpoint. It interacts with an actual test server which has been built as a platform simulator to support this notebook.


## Setup

Here we just import the libraries that will be needed in this notebook, define some utility functions and constants.

In [178]:
import requests
import json
import jwt
import base64
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicNumbers
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from IPython.display import display, Markdown, HTML, Javascript

def decode_int(b64value):
    return int.from_bytes(base64.urlsafe_b64decode(b64value), byteorder='big')

# for concise code, return full claim prefixed by ims
def fc(claim):
    return "http://imsglobal.org/lti/{0}".format(claim)

def md(mdt):
    display(Markdown(mdt))
    
md_buffer = ''

def mdb(mdt=None):
    global md_buffer
    if mdt:
        md_buffer = md_buffer + '\n' + mdt
    else:
        md(md_buffer)
        md_buffer = ''

# This notebook queries an actual test server. It needs its location:
platform_url='http://localhost:5000'


First we need to get a new tool deployment from the server for this notebook instance to use.
Each tool must have a client_id and a private key that will be used to interact with the platform services and send messages back to the platform.
It also needs the keyset URL that exposes the platform public keys needed to validate the incoming messages.

While this information is required for each tool, how it is obtained by the tool is NOT currently part of the LTI specifications.

In [163]:
r = requests.get(platform_url + '/newtool')
tool_info = json.loads(r.text)

md('### Tool information')
md('Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.')
md('```json\n'+ json.dumps(tool_info, indent=4)+'```')

### Tool information

Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.

```json
{
    "client_id": "0",
    "deployment_id": "deployment_0",
    "key": {
        "e": "AQAB",
        "kty": "RSA",
        "use": "sig",
        "alg": "RS256",
        "d": "A3p4hVUZbViDnx0STEtz1jnZ_gtjkl9yMX3288QSvdjUvSsv8gIpSJQh3z8caZ5-HrQ7k8OFsMZcQkWPLa9O8XmZ5tTViJ3iU4-w2hrX5jdYbgLjMJI4cmXlyw1KNUt1xQs5bjoG_d7XRPdxGC-BhqbJNalHQ94Tek8CiLnSBv3CNEd4l9scHc88yfR7qfyJzMhPviugetGhPoN7VTW3VTffCUDk52u31sqEid1LzQI393MmV7mHZ9eu_baGvPZ86fr65uBQBxfMrW66SE0v5XZ6cXRUbK_74XLJg4ioQU77gmwOm1wBUE5EfI_xNWChPdRp_eCZ2YTX4A1a2D_DAQ=="
    }
}```

### Getting the public keys from the platform

In order to validate the various messages we will receive from the platform, we need to get the key sets. We'll also transform the keys to the `PEM` format that is used by the `jwt` module to decode the messages.

The test platform exposes its keys in a keyset format at a well-know location (.well-known/jwks.json). Other platform might just communicate the keyset url as part of the tool deployment information.

In [164]:
keyset = json.loads(requests.get(platform_url + '/.well-known/jwks.json').text)

md('#### Platform keyset')
md('```json\n'+ json.dumps(keyset, indent=4)+'```')

platform_keys = {}

# let's transform as a map for ease of use, and just the PEM because this is what is used by JWT lib
for key in keyset['keys']:
    public_key = RSAPublicNumbers(decode_int(key['e']), decode_int(key['n'])).public_key(default_backend())
    pem = public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)
    platform_keys[key['kid']] = pem
    


#### Platform keyset

```json
{
    "keys": [
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "vU8vT0BbopWi3MHJJYgExZ9PtJH1DQTeq_N2GEah05KFuRJ6XQaBQoaZ6znrwM0r-AaOp3Yo727oPgduAQMEWB3T72HRA8fJK3eIb0Q7wQG5h0ON3ZiCElARPCV0_DqNgvxUjfZXrQXRIyRMUprY1_Y36lkGo337HUQxn7s1ClehNMp6tqPqaO_G8xlmRhTRVqQgd84UObpEwoCOcGeJ942sdP1xBggNfBef4sE9ebYahhtSJyNbEw1pAF6_wEcHnn9mmd7_tCuSQeAD3IQkxyiHZQrqxbVNTc4wRKo4woXz6mOQT8Wiq8eXqMpSHtrzqRII2XjiqEPbyJHGx9LF3Q==",
            "kid": "1517695759_0"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "7FqiFCafD2OeuRokS6LMv9UheuYnkzB_N_JImUQp2juT0yc10Z_g1Dfo_opPLI7SK6nA9VzmL6ZKxasY6e2vOppJ0Fu8-gaXOL2Jvvr7cQLB1PxpLJfRF6tmASX0A-vdG_5fgvw0O1BRdUl2O2O2DpnSFDS9CGVt3COzbZp5v6EEuy9_PgmA64-cmt7Sq3F4mUbc5IO2mpnJm4ns_2JGAYZe0qGGFqFljI8wSIkBnlfaq1uiObSorimzGSwdgYNU9UxWLC_tZPON3rTyI3OQpArJ8hbzdHDP2gSwJ8pj7TUsmjV2xAvlXC77R8N8bVsBNuWQS7hvDnGu5hTCKRF1cw==",
            "kid": "1517695759_1"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "0vT_Br6OuzGPouAv6w7jAr2-Pe9O_UwFxWhnt3h54BfmvBEY6CGttSJaY_3H1Ymu5RKCd95utWI_2YFeI5kRnJPGTLOvgQ0_3-0VbXGQ_DWCYWOufMdW55viFZFWkcLwjxA8vqssF86MbOS8GOk-gUa2hixbiIoCZLLNyqmKu6og-uf7yeuMqm7AlMnNEf-nuT2q91LJeB6HHUtOJh8me63qCstB0kN_cSGeLSdYJ6U6flsKRZN0b75zTeAhNP4N8GUgczt481UOR26ANkrbCtaO3XqKAsUT3W2S7ticfgqojUH3OOifvldme8H1L5xrznQIwFRq7_DnEwjWMxqMjw==",
            "kid": "1517695759_2"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "pFHfq1YRoDK9jcqeVjLW-6wIhnOGxM1XQmTvPToZ4qoAaSe2HOnTVoMVoO1snCZUaJZJitFCkrQsplKxMeh1L2Y9whQ31MXN2LM3ORoY_MQHb8jhK_IXykVyRnYF2L_yyKc1Wan11vVhzPhem3QzxOTVQiN8YLX2pqwaG3fIoqvznh9SILI3lNUYiBpVtNHIvg0MPSqZe7J4We8J5YIXL12_KoBRdbLetJHd8jI7XhC1GTzTEGicg7Y6AOJw8UqtipGam10Y9z_pdLnMOIIq5yzMey3_ksJ9qqWH6dB2kn8jJFmxF9htCgsGXfSTmkUIgdUSpSIdRzMYAG4myTD1Vw==",
            "kid": "1517695759_3"
        }
    ]
}```

## Deep Linking - Creating a Link
This section will use the deep linking specification to create a Resource Link to the platform. That resource link will be gradable and used in the following sections of that notebook.

[Deep Linking](https://www.imsglobal.org/specs/lticiv1p0) is a **UI flow**, it is an important piece that is sometimes missed on 1st reading. The user is redirected from the platform to the tool to pick or create one or multiple piece of content (often, LTI links), and the the tool redirects the UI back to the tool with the actual selection (or an empty payload if nothing was picked or created).

So there are 2 messages:

1. `ContentItemRequestSelection` from the platform to the tool to start the picking/create session. This is a typical platform launch that contains the context and the user information, and what kind of content items may be created in this flow (for example, this flow might indicate it only wants LTI links).
1. `ContentItemSelection` from the tool back to the plaform using the `content_item_return_url` provided in the request.

Once a tool is added to a course, usually the first launch from the platform will be a Deep Linking request.

<img src="assets/lti_advantage_deeplinking.png" width="60%">

### Setup: Getting a Deep Linking Request

This notebook is not a tool actually launched by the platform, so the test platform as way to give us the token that it would include in an actual HTTP POST request, so we can build a mock POST request including the parameter `post_data`.

In [168]:
r = requests.get("{}/tool/{}/cisr".format(platform_url, tool_info['client_id']))

post_data = {
    'id_token':r.text
}

md('`id_token='+ r.text+'`')

`id_token=eyJraWQiOiIxNTE3Njk1NzU5XzAiLCJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJodHRwOi8vaW1zZ2xvYmFsLm9yZy9sdGkvbWVzc2FnZV90eXBlIjoiQ29udGVudEl0ZW1TZWxlY3Rpb25SZXF1ZXN0IiwiZXhwIjoxNTE3Njk1OTE2LCJmYW1pbHlfbmFtZSI6IkNvbmNoXHUwMGZhciIsIm5vbmNlIjoiMWI3NjQ1YzAtMDkyZi0xMWU4LWEyMmMtYzQ4ZThmZmI3ODU3IiwibmFtZSI6IkNocmlzdGFsIENvbmNoXHUwMGZhciIsImlhdCI6MTUxNzY5NTg1NiwiaHR0cDovL2ltc2dsb2JhbC5vcmcvbHRpL2RlcGxveW1lbnRfaWQiOiJkZXBsb3ltZW50XzAiLCJhdWQiOiIwIiwiaHR0cDovL2ltc2dsb2JhbC5vcmcvbHRpL2xhdW5jaF9wcmVzZW50YXRpb24iOnsiZG9jdW1lbnRfdGFyZ2V0IjoiaWZyYW1lIiwicmV0dXJuX3VybCI6Ii90b29sL2VmYzhiMjNjLTA5MmUtMTFlOC1hMjJjLWM0OGU4ZmZiNzg1Ny9jaXNyIn0sImVtYWlsIjoiQ2hyaXN0YWwuQ29uY2hcdTAwZmFyQGV4YW1wbGUuY29tIiwiaHR0cDovL2ltc2dsb2JhbC5vcmcvbHRpL2NvbnRleHQiOnsidGl0bGUiOiJMVEkgQm9vdGNhbXAgQ291cnNlIiwiaWQiOiJlZmM4YjIzYy0wOTJlLTExZTgtYTIyYy1jNDhlOGZmYjc4NTciLCJsYWJlbCI6IkxUSSBCb290Y2FtcCBDb3Vyc2UiLCJ0eXBlIjpbIkNvdXJzZVNlY3Rpb24iXX0sImdpdmVuX25hbWUiOiJDaHJpc3RhbCIsInN1YiI6IkxUSUJDVV8zIiwiaHR0cDovL2ltc2dsb2JhbC5vcmcvbHRpL3JvbGVzIjpbImh0dHA6Ly9wdXJsLmltc2dsb2JhbC5vcmcvdm9jYWIvbGlzL3YyL21lbWJlcnNoaXAjSW5zdHJ1Y3RvciJdLCJodHRwOi8vaW1zZ2xvYmFsLm9yZy9sdGkvdmVyc2lvbiI6IjEuMy4wIiwiaXNzIjoiaHR0cDovL2xvY2FsaG9zdDo1MDAwIiwiaHR0cDovL2ltc2dsb2JhbC5vcmcvbHRpL2RlZXBfbGlua2luZ19yZXF1ZXN0Ijp7ImF1dG9fY3JlYXRlIjp0cnVlLCJhY2NlcHRfbXVsdGlwbGUiOnRydWUsImRhdGEiOiJvcD0zMjEmdj00NCIsImFjY2VwdF9tZWRpYV90eXBlcyI6WyJhcHBsaWNhdGlvbi92bmQuaW1zLmx0aS52MS5sdGlsaW5rIl0sImFjY2VwdF9wcmVzZW50YXRpb25fZG9jdW1lbnRfdGFyZ2V0cyI6WyJpZnJhbWUiLCJ3aW5kb3ciXX0sImh0dHA6Ly9pbXNnbG9iYWwub3JnL2x0aS90b29sX3BsYXRmb3JtIjp7Im5hbWUiOiJMVEkgQm9vdGNhbXAgUGxhdGZvcm0iLCJndWlkIjoibHRpYmNfYXRfMTUxNzY5NTc2MCJ9fQ.Xcj_RDaInTRToEWuBUufslT2LTbBAF5gHhM4g6i856Oltpu2IB9UvvBrlylQO5KH8e9kn-T4Kyrr8BMlIPdyd5tKTWiMC89bMMi64E5JZNHQ7diaCUdvPiySm9ybglAEofaexyzZxjERBExKlSJwdOyMmBRxlVTrjBeelj7JuCuealN_0XVoqcM9Tbnnbg4lLwZmeE_2CABUZSm6n8WPl459UWAAL73AuS-OvtzrlJR-srbkMOx9gvkuJl0HI6eULfBkep-W-FJJjMjM7X06kusXZbw8ldD5yJvUrIOZkrbnxxCe1phl7ZTxEfCU9aYw8f1JuBKZLdc3n5qmVSgg3A`

### Task 1: Verify the JWT is properly signed

The first thing before to display to the user the picker/authoring interface to create the link is to validate this request is properly signed. This is done by decoding the JWT using public key from the platform.


In [169]:
# Let's get the kid so we can get the proper public key
encoded_jwt = post_data['id_token']
jwt_headers = jwt.get_unverified_header(encoded_jwt)

content_item_message = jwt.decode(encoded_jwt, 
                                  platform_keys[jwt_headers['kid']], 
                                  jwt_headers['alg'],
                                  audience = tool_info['client_id'])

md('#### Message properly signed! Decoded ContentItemSelectionRequest message:')
md('```json\n'+ json.dumps(content_item_message, indent=4)+'```')


#### Message properly signed! Decoded ContentItemSelectionRequest message:

```json
{
    "given_name": "Christal",
    "http://imsglobal.org/lti/version": "1.3.0",
    "sub": "LTIBCU_3",
    "http://imsglobal.org/lti/context": {
        "title": "LTI Bootcamp Course",
        "type": [
            "CourseSection"
        ],
        "id": "efc8b23c-092e-11e8-a22c-c48e8ffb7857",
        "label": "LTI Bootcamp Course"
    },
    "http://imsglobal.org/lti/roles": [
        "http://purl.imsglobal.org/vocab/lis/v2/membership#Instructor"
    ],
    "email": "Christal.Conch\u00far@example.com",
    "family_name": "Conch\u00far",
    "iat": 1517695856,
    "http://imsglobal.org/lti/deep_linking_request": {
        "accept_media_types": [
            "application/vnd.ims.lti.v1.ltilink"
        ],
        "accept_multiple": true,
        "auto_create": true,
        "data": "op=321&v=44",
        "accept_presentation_document_targets": [
            "iframe",
            "window"
        ]
    },
    "exp": 1517695916,
    "nonce": "1b7645c0-092f-11e8-a22c-c48e8ffb7857",
    "http://imsglobal.org/lti/deployment_id": "deployment_0",
    "aud": "0",
    "http://imsglobal.org/lti/message_type": "ContentItemSelectionRequest",
    "http://imsglobal.org/lti/tool_platform": {
        "guid": "ltibc_at_1517695760",
        "name": "LTI Bootcamp Platform"
    },
    "iss": "http://localhost:5000",
    "name": "Christal Conch\u00far",
    "http://imsglobal.org/lti/launch_presentation": {
        "document_target": "iframe",
        "return_url": "/tool/efc8b23c-092e-11e8-a22c-c48e8ffb7857/cisr"
    }
}```

### Task 2: extract the information needed to render the selector/authoring UI

If this is the first launch for the user or the course, as a tool you may prompt the user for setup information, including account linking or course setup. Ultimately the user will see the authoring or picking interface that will allow her to create or select the content items to be added to the course.

Some key attributes of the `ContentItemSelectionRequest` will drive the experience:

In [174]:
# fc(claim) prefix the claim with http://imsglobal.org/lti/
mdb('1. What is the current course id? {0}'.format(content_item_message[fc('context')]['id']))
mdb('1. What is the current user id? {0}'.format(content_item_message['sub']))
is_instructor = len(list(filter(lambda x: 'instructor' in x.lower(), content_item_message[fc('roles')])))>0
mdb('1. Is this user an instructor? {0}'.format(is_instructor))
deep_linking_claim = content_item_message[fc('deep_linking_request')]
mdb('1. What kind of content item can be created? {0}'.format(deep_linking_claim['accept_media_types']))
mdb('1. Can I return more than one items to be added? {0}'.format(deep_linking_claim['accept_multiple']))
mdb('1. Will the user be prompted before to actually save the items? {0}'.format(not deep_linking_claim['auto_create']))
mdb('1. Where should I redirect the browser too when done? {0}'.format(content_item_message[fc('launch_presentation')]['return_url']))
mdb('1. Is there any data I must pass back to platform when I return? {0}'.format('data' in deep_linking_claim))
mdb()


1. What is the current course id? efc8b23c-092e-11e8-a22c-c48e8ffb7857
1. What is the current user id? LTIBCU_3
1. Is this user an instructor? True
1. What kind of content item can be created? ['application/vnd.ims.lti.v1.ltilink']
1. Can I return more than one items to be added? True
1. Will the user be prompted before to actually save the items? False
1. Where should I redirect the browser too when done? /tool/efc8b23c-092e-11e8-a22c-c48e8ffb7857/cisr
1. Is there any data I must pass back to platform when I return? True

### Task 3: building the response token

After the end of the interaction, so user is sent back to the platform throught a browser redirection using an HTTP POST containing the JWT `ContentItemResponse` message. In this case, we will return 2 LTI links, one being graded (since the request supports multiple content items).

Here we're creating the actual response token.

In [195]:
## First let's create our 2 content items
## Note that the URLs are phony as for now there is now way to launch in the notebook anyway...
simple_link =  {
      "mediaType": "application/vnd.ims.lti.v1.ltilink",
      "url": "http://lti.bootcamp/item111",
      "presentationDocumentTarget": "iframe",
      "title": "A simple content item",
      "text": "Some long text",
      "icon": {
        "url": "http://lti.example.com/image.jpg",
        "width": 100,
        "height": 100
      },
      "custom": {
        "key1": "some value"
      }
}

assignment_link =  {
      "mediaType": "application/vnd.ims.lti.v1.ltilink",
      "url": "http://lti.bootcamp/item111",
      "presentationDocumentTarget": "iframe",
      "title": "An assignment",
      "text": "Chemical lab sim",
      "icon": {
        "url": "http://lti.example.com/image.jpg",
        "width": 100,
        "height": 100
      },
      "custom": {
        "lab": "sim3a",
        "level": "easy"
      },
      "lineItem": {
        "scoreMaximum": 87,
        "label": "Chapter 12 quiz",
        "resourceId": "xyzpdq1234",
        "tag": "originality"
      }
}

### Task 4: redirect the user back to the platform with the content item selection

Now that we the response token, let's do the actual HTML POST redirection to the platform. Note that because the platform supports `autocreate` there will be no prompt. The 2 items will be added directly to the course.


In [193]:
# Let's start by adding the JWS security claims
content_item_response = {
    'iss': tool_info['client_id'] ,
    'aud': content_item_message['iss']
}

autosubmit_js = """
                var ltiForm = $('<form>');                
                ltiForm.attr('action', 'http://localhost:5000');
                ltiForm.attr('method', 'get');
                ltiForm.attr('target', 'deeplinking_frame');
                $('#deeplinking_frame').before(ltiForm);
                ltiForm.submit();
                ltiForm.remove();
                """

display(HTML('<iframe id="deeplinking_frame" name="deeplinking_frame" style="height: 300px; width:100%"></iframe>'))
display(Javascript(data=autosubmit_js, 
                   lib="https://code.jquery.com/jquery-3.3.1.min.js"))

<IPython.core.display.Javascript object>